In [1]:
from lxml import html
import requests
import json
from datetime import date
from datetime import datetime
from pymongo import MongoClient
from pprint import pprint

In [2]:
url = 'https://lenta.ru/'
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

response = requests.get(url, headers = headers)

In [3]:
response.url

'https://lenta.ru/'

In [4]:
dom = html.fromstring(response.text)

In [5]:
#card-mini
#card-big
#card-feature
#//div[contains(@class, 'card-big') or contains(@class, 'card-mini') or contains(@class, 'card-feature')]
news = dom.xpath("//a[contains(@class, 'card-big') or contains(@class, 'card-mini') or contains(@class, 'card-feature')]")
now = datetime.now()

In [6]:
months = {'1': 'января', '2': 'февраля', '3': 'марта', '4': 'апреля', '5': 'мая', '6': 'июня', '7': 'июля',
          '8': 'августа', '9': 'сентября', '10': 'октября', '11': 'ноября', '12': 'декабря'}

In [7]:
def get_ext(url):
    headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

    response = requests.get(url, headers = headers)
    return response.url.split('/')[2], response.url

In [8]:
news_list = list()

for piece in news:
    piece_text = piece.xpath(".//h3/text()")
    if len(piece_text) == 0:
        piece_text = piece.xpath(".//span/text()")
    piece_text = piece_text[0]
    piece_link = piece.xpath(".//@href")[0]
    piece_source = 'lenta.ru'
    try:
        piece_time = piece.xpath(".//time/text()")[0]
        if len(piece_time) == 5:
            piece_time = f'{piece_time}, {now.day} {months[str(now.month)]} {now.year}'
    except:
        piece_time = piece_link.split('/')[2:5]
        piece_time = piece_time[2] + ' ' + months[piece_time[1]] + ' ' + piece_time[0]
        #piece_time = 'None'
    if 'extlink' in piece_link:
        piece_source, piece_link = get_ext(piece_link)
    else:
        piece_link = 'https://lenta.ru' + piece_link
    
    
    news_list.append({'source': piece_source, 'title': piece_text, 'link': piece_link, 'date': piece_time})

In [9]:
client = MongoClient()

In [10]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [11]:
client['db_lenta']

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_lenta')

In [12]:
client.list_database_names()

['admin', 'config', 'db_lenta', 'db_test', 'local']

In [13]:
db = client.db_lenta

In [14]:
db.news.drop()

In [15]:
for piece in news_list:
    db.news.insert_one(piece)

In [16]:
for piece in db.news.find():
    pprint(piece)

{'_id': ObjectId('63b0c7e8d4fb2cdf69e9779e'),
 'date': '02:01, 1 января 2023',
 'link': 'https://lenta.ru/news/2023/01/01/putin_obrashenie/',
 'source': 'lenta.ru',
 'title': 'Путин обратился к россиянам с новогодним поздравлением'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9779f'),
 'date': '02:35, 1 января 2023',
 'link': 'https://lenta.ru/news/2023/01/01/suhpay/',
 'source': 'lenta.ru',
 'title': 'Военкор показал новогодний сухпаек для бойцов на передовой'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e977a0'),
 'date': '02:24, 1 января 2023',
 'link': 'https://lenta.ru/news/2023/01/01/_kndr_/',
 'source': 'lenta.ru',
 'title': 'Ким Чен Ын назвал зону поражения тактических ядерных ракет КНДР'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e977a1'),
 'date': '02:17, 1 января 2023',
 'link': 'https://lenta.ru/news/2023/01/01/prognoz/',
 'source': 'lenta.ru',
 'title': 'Иммунолог дал россиянам прогноз на 2023 год'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e977a2'),
 'date': '02:09, 1 января 2023',
 'link': 'https:

          'Новый год'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9782a'),
 'date': '00:01, 27 июля 2022',
 'link': 'https://lenta.ru/articles/2022/07/27/issandross/',
 'source': 'lenta.ru',
 'title': 'Дальше сами.'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9782b'),
 'date': '20:34, 31 декабря 2022',
 'link': 'https://lenta.ru/news/2022/12/31/ubiystvo_pod_mostom/',
 'source': 'lenta.ru',
 'title': 'Сотрудника МАДИ убили под Парящим мостом в «Зарядье»'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9782c'),
 'date': '00:02, 27 сентября 2022',
 'link': 'https://lenta.ru/articles/2022/09/27/parkchanwookinterview/',
 'source': 'lenta.ru',
 'title': '«Мне больше не нужны насилие и обнаженка»'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9782d'),
 'date': '20:23, 31 декабря 2022',
 'link': 'https://lenta.ru/news/2022/12/31/tsel_udara/',
 'source': 'lenta.ru',
 'title': 'Власти Первомайска назвали цель ударов ВСУ из HIMARS по городу'}
{'_id': ObjectId('63b0c7e8d4fb2cdf69e9782e'),
 'date': '20:11, 31 декабря 2022',
 'l